In [3]:
import pandas as pd
import twint
import nest_asyncio
from datetime import date

def tweet_scrape(search_term, ):
    nest_asyncio.apply()
    c = twint.Config()
    c.Lang = "en"
    c.Since = 'datetime.today()'
    c.Search = [search_term]
    c.Pandas = True
    twint.run.Search(c)
    Tweets_df = twint.storage.panda.Tweets_df
    return Tweets_df



2022-07-22


In [11]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def vader_scores(tweet):
    """
    Run VADER sentiment analysis and return  values
    """
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(tweet)
    return list(vs.values())


def score_columns():
    """
    Apply those values to the tweets_df and then seperate them into distinct columns
    """
    Tweets_df['Sentiment'] = Tweets_df['tweet'].map(vader_scores)
    Tweets_df[['negative','neutral','positive', 'compound']] = Tweets_df['Sentiment'].tolist()
    Tweets_df.drop(columns='Sentiment', inplace=True)

score_columns()


In [12]:
from google.cloud import bigquery

Tweets_df.head()

def load_bq(dataframe, table):
    client = bigquery.Client()
    df = dataframe
    job = client.load_table_from_dataframe(df, table)
    job.result()

load_bq(Tweets_df,'deb-01-346001.vibe_shift.hydroflask')

/home/jessgschu/projects/vibe-shift/venv/lib/python3.8/site-packages/google/cloud/bigquery/_pandas_helpers.py:558: UserWarning: Pyarrow could not determine the type of columns: cashtags, reply_to.
  warnings.warn(
